# LLMCompiler

This notebook shows how to implement [LLMCompiler, by Kim, et. al](https://arxiv.org/abs/2312.04511) in LangGraph.

LLMCompiler is an agent architecture designed to **speed up** the execution of agentic tasks by eagerly-executed tasks within a DAG. It also saves costs on redundant token usage by reducing the number of calls to the LLM. Below is an overview of its computational graph:

![LLMCompiler Graph](./img/llm-compiler.png)

It has 3 main components:

1. Planner: stream a DAG of tasks.
2. Task Fetching Unit: schedules and executes the tasks as soon as they are executable
3. Joiner: Responds to the user or triggers a second plan


This notebook walks through each component and shows how to wire them together using LangGraph. The end result will leave a trace [like the following](https://smith.langchain.com/public/218c2677-c719-4147-b0e9-7bc3b5bb2623/r).


**First,** install the dependencies, and set up LangSmith for tracing to more easily debug and observe the agent.

In [1]:
%pip install -U --quiet langchainhub langchain_openai langsmith langgraph langchain numexpr langchain_community

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import getpass


def _get_pass(var: str):
    if var not in os.environ:
        os.environ[var] = getpass.getpass(f"{var}: ")


# Optional: Debug + trace calls using LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "True"
os.environ["LANGCHAIN_PROJECT"] = "LLMCompiler"
_get_pass("LANGCHAIN_API_KEY")
_get_pass("OPENAI_API_KEY")

## Part 1: Tools

We'll first define the tools for the agent to use in our demo. We'll give it the class search engine + calculator combo.

If you don't want to sign up for tavily, you can replace it with the free [DuckDuckGo](https://python.langchain.com/docs/integrations/tools/ddg).

In [3]:
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

# Imported from the https://github.com/langchain-ai/langgraph/tree/main/examples/plan-and-execute repo
from math_tools import get_math_tool

_get_pass("TAVILY_API_KEY")

calculate = get_math_tool(ChatOpenAI(model="gpt-4o"))
search = TavilySearchResults(
    max_results=1,
    description='tavily_search_results_json(query="the search query") - a search engine.',
)

tools = [search, calculate]

/workspaces/langgraph/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: LangChain has introduced a method called `with_structured_output` that is available on ChatModels capable of tool calling. You can read more about the method here: https://python.langchain.com/docs/modules/model_io/chat/structured_output/ Please follow our extraction use case documentation for more guidelines on how to do information extraction with LLMs. https://python.langchain.com/docs/use_cases/extraction/. If you notice other issues, please provide feedback here: https://github.com/langchain-ai/langchain/discussions/18154
  warn_deprecated(


In [4]:
calculate.invoke(
    {
        "problem": "What's the temp of sf + 5?",
        "context": ["Thet empreature of sf is 32 degrees"],
    }
)

'37'

In [5]:
from langchain_openai import ChatOpenAI
from youtube_tools import get_youtube_parser_tool  # Adjust the import based on your file structure

# Initialize the language model
llm = ChatOpenAI(model="gpt-4o")

# Get the youtube_parser tool
youtube_parser_tool = get_youtube_parser_tool(llm)

# Define the YouTube video URL
video_url = "https://www.youtube.com/watch?v=dQw4w9WgXcQ"

# Call the tool with the video URL
result = youtube_parser_tool.func(video_url=video_url)

# Print the result
print(result)

DATA MODEL = video_url='https://www.youtube.com/watch?v=dQw4w9WgXcQ' context=None
{'url': 'https://www.youtube.com/watch?v=dQw4w9WgXcQ', 'title': 'Sample Title', 'description': 'Sample description of the video.', 'views': '12345', 'likes': '678', 'dislikes': '90', 'comments': [{'author': 'User1', 'text': 'Sample comment 1'}, {'author': 'User2', 'text': 'Sample comment 2'}]}


In [6]:
tools.append(youtube_parser_tool)

for t in tools:
    print(t)

description='tavily_search_results_json(query="the search query") - a search engine.' max_results=1
name='math' description='math(problem: str, context: Optional[list[str]]) -> float:\n - Solves the provided math problem.\n - `problem` can be either a simple math problem (e.g. "1 + 3") or a word problem (e.g. "how many apples are there if there are 3 apples and 2 apples").\n - You cannot calculate multiple expressions in one call. For instance, `math(\'1 + 3, 2 + 4\')` does not work. If you need to calculate multiple expressions, you need to call them separately like `math(\'1 + 3\')` and then `math(\'2 + 4\')`\n - Minimize the number of `math` actions as much as possible. For instance, instead of calling 2. math("what is the 10% of $1") and then call 3. math("$1 + $2"), you MUST call 2. math("what is the 110% of $1") instead, which will reduce the number of math actions.\n - You can optionally provide a list of strings as `context` to help the agent solve the problem. If there are mul

# Part 2: Planner


Largely adapted from [the original source code](https://github.com/SqueezeAILab/LLMCompiler/blob/main/src/llm_compiler/output_parser.py), the planner  accepts the input question and generates a task list to execute.

If it is provided with a previous plan, it is instructed to re-plan, which is useful if, upon completion of the first batch of tasks, the agent must take more actions.

The code below composes constructs the prompt template for the planner and composes it with LLM and output parser, defined in [output_parser.py](./output_parser.py). The output parser processes a task list in the following form:

```plaintext
1. tool_1(arg1="arg1", arg2=3.5, ...)
Thought: I then want to find out Y by using tool_2
2. tool_2(arg1="", arg2="${1}")'
3. join()<END_OF_PLAN>"
```

The "Thought" lines are optional. The `${#}` placeholders are variables. These are used to route tool (task) outputs to other tools.

In [7]:
from typing import Sequence

from langchain_core.language_models import BaseChatModel
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableBranch
from langchain_core.tools import BaseTool
from langchain_core.messages import (
    BaseMessage,
    FunctionMessage,
    HumanMessage,
    SystemMessage,
)

from output_parser import LLMCompilerPlanParser, Task
from langchain import hub
from langchain_openai import ChatOpenAI


prompt = hub.pull("wfh/llm-compiler")
print(prompt.pretty_print())

================================ System Message ================================

Given a user query, create a plan to solve it with the utmost parallelizability. Each plan should comprise an action from the following {num_tools} types:
{tool_descriptions}
{num_tools}. join(): Collects and combines results from prior actions.

 - An LLM agent is called upon invoking join() to either finalize the user query or wait until the plans are executed.
 - join should always be the last action in the plan, and will be called in two scenarios:
   (a) if the answer can be determined by gathering the outputs from tasks to generate the final response.
   (b) if the answer cannot be determined in the planning phase before you execute the plans. Guidelines:
 - Each action described above contains input/output types and description.
    - You must strictly adhere to the input and output types for each action.
    - The action descriptions contain the guidelines. You MUST strictly follow those guideline

In [8]:
def create_planner(
    llm: BaseChatModel, tools: Sequence[BaseTool], base_prompt: ChatPromptTemplate
):
    tool_descriptions = "\n".join(
        f"{i+1}. {tool.description}\n"
        for i, tool in enumerate(
            tools
        )  # +1 to offset the 0 starting index, we want it count normally from 1.
    )
    planner_prompt = base_prompt.partial(
        replan="",
        num_tools=len(tools)
        + 1,  # Add one because we're adding the join() tool at the end.
        tool_descriptions=tool_descriptions,
    )
    replanner_prompt = base_prompt.partial(
        replan=' - You are given "Previous Plan" which is the plan that the previous agent created along with the execution results '
        "(given as Observation) of each plan and a general thought (given as Thought) about the executed results."
        'You MUST use these information to create the next plan under "Current Plan".\n'
        ' - When starting the Current Plan, you should start with "Thought" that outlines the strategy for the next plan.\n'
        " - In the Current Plan, you should NEVER repeat the actions that are already executed in the Previous Plan.\n"
        " - You must continue the task index from the end of the previous one. Do not repeat task indices.",
        num_tools=len(tools) + 1,
        tool_descriptions=tool_descriptions,
    )

    def should_replan(state: list):
        # Context is passed as a system message
        return isinstance(state[-1], SystemMessage)

    def wrap_messages(state: list):
        return {"messages": state}

    def wrap_and_get_last_index(state: list):
        next_task = 0
        for message in state[::-1]:
            if isinstance(message, FunctionMessage):
                next_task = message.additional_kwargs["idx"] + 1
                break
        state[-1].content = state[-1].content + f" - Begin counting at : {next_task}"
        return {"messages": state}

    return (
        RunnableBranch(
            (should_replan, wrap_and_get_last_index | replanner_prompt),
            wrap_messages | planner_prompt,
        )
        | llm
        | LLMCompilerPlanParser(tools=tools)
    )

In [9]:
llm = ChatOpenAI(model="gpt-4o")
# This is the primary "agent" in our application
planner = create_planner(llm, tools, prompt)

In [10]:
example_question = "What's the temperature in SF raised to the 3rd power?"

for task in planner.stream([HumanMessage(content=example_question)]):
    print(task["tool"], task["args"])
    print("---")

description='tavily_search_results_json(query="the search query") - a search engine.' max_results=1 {'query': 'current temperature in San Francisco'}
---
name='math' description='math(problem: str, context: Optional[list[str]]) -> float:\n - Solves the provided math problem.\n - `problem` can be either a simple math problem (e.g. "1 + 3") or a word problem (e.g. "how many apples are there if there are 3 apples and 2 apples").\n - You cannot calculate multiple expressions in one call. For instance, `math(\'1 + 3, 2 + 4\')` does not work. If you need to calculate multiple expressions, you need to call them separately like `math(\'1 + 3\')` and then `math(\'2 + 4\')`\n - Minimize the number of `math` actions as much as possible. For instance, instead of calling 2. math("what is the 10% of $1") and then call 3. math("$1 + $2"), you MUST call 2. math("what is the 110% of $1") instead, which will reduce the number of math actions.\n - You can optionally provide a list of strings as `context`

In [11]:
example_question = "What is the description of this video https://www.youtube.com/watch?v=dQw4w9WgXcQ by Krish naik on datascience?"

for task in planner.stream([HumanMessage(content=example_question)]):
    print(task["tool"], task["args"])
    print("---")

name='youtube_parser' description='youtube_parser(video_url: str, context: Optional[list[str]]) -> dict:\n - Extracts metadata and comments from the provided YouTube video URL.\n - `video_url` should be a valid YouTube video URL.\n - You can optionally provide a list of strings as `context` to help the agent extract specific information.\n - The returned dictionary contains video title, description, views, likes, dislikes, and comments.\n - Minimize the number of `youtube_parser` actions as much as possible.' args_schema=<class 'pydantic.v1.main.youtube_parserSchema'> func=<function get_youtube_parser_tool.<locals>.parse_youtube_video at 0x71fbd8273760> {'video_url': 'https://www.youtube.com/watch?v=dQw4w9WgXcQ', 'context': ['Krish Naik', 'datascience']}
---
join ()
---


In [13]:
example_question = "What is the description of the latest youtube video by Krish naik on datascience?"

for task in planner.stream([HumanMessage(content=example_question)]):
    print(task["tool"], task["args"])
    print("---")

description='tavily_search_results_json(query="the search query") - a search engine.' max_results=1 {'query': 'latest youtube video by Krish Naik on datascience'}
---
name='youtube_parser' description='youtube_parser(video_url: str, context: Optional[list[str]]) -> dict:\n - Extracts metadata and comments from the provided YouTube video URL.\n - `video_url` should be a valid YouTube video URL.\n - You can optionally provide a list of strings as `context` to help the agent extract specific information.\n - The returned dictionary contains video title, description, views, likes, dislikes, and comments.\n - Minimize the number of `youtube_parser` actions as much as possible.' args_schema=<class 'pydantic.v1.main.youtube_parserSchema'> func=<function get_youtube_parser_tool.<locals>.parse_youtube_video at 0x71fbd8273760> {'video_url': '$1'}
---
join ()
---


## 3. Task Fetching Unit

This component schedules the tasks. It receives a stream of tools of the following format:

```typescript
{
    tool: BaseTool,
    dependencies: number[],
}
```


The basic idea is to begin executing tools as soon as their dependencies are met. This is done through multi-threading. We will combine the task fetching unit and executor below:

![diagram](./img/diagram.png)

In [14]:
from typing import Any, Union, Iterable, List, Tuple, Dict
from typing_extensions import TypedDict
import re

from langchain_core.runnables import (
    chain as as_runnable,
)

from concurrent.futures import ThreadPoolExecutor, wait
import time


def _get_observations(messages: List[BaseMessage]) -> Dict[int, Any]:
    # Get all previous tool responses
    results = {}
    for message in messages[::-1]:
        if isinstance(message, FunctionMessage):
            results[int(message.additional_kwargs["idx"])] = message.content
    return results


class SchedulerInput(TypedDict):
    messages: List[BaseMessage]
    tasks: Iterable[Task]


def _execute_task(task, observations, config):
    tool_to_use = task["tool"]
    if isinstance(tool_to_use, str):
        return tool_to_use
    args = task["args"]
    try:
        if isinstance(args, str):
            resolved_args = _resolve_arg(args, observations)
        elif isinstance(args, dict):
            resolved_args = {
                key: _resolve_arg(val, observations) for key, val in args.items()
            }
        else:
            # This will likely fail
            resolved_args = args
    except Exception as e:
        return (
            f"ERROR(Failed to call {tool_to_use.name} with args {args}.)"
            f" Args could not be resolved. Error: {repr(e)}"
        )
    try:
        return tool_to_use.invoke(resolved_args, config)
    except Exception as e:
        return (
            f"ERROR(Failed to call {tool_to_use.name} with args {args}."
            + f" Args resolved to {resolved_args}. Error: {repr(e)})"
        )


def _resolve_arg(arg: Union[str, Any], observations: Dict[int, Any]):
    # $1 or ${1} -> 1
    ID_PATTERN = r"\$\{?(\d+)\}?"

    def replace_match(match):
        # If the string is ${123}, match.group(0) is ${123}, and match.group(1) is 123.

        # Return the match group, in this case the index, from the string. This is the index
        # number we get back.
        idx = int(match.group(1))
        return str(observations.get(idx, match.group(0)))

    # For dependencies on other tasks
    if isinstance(arg, str):
        return re.sub(ID_PATTERN, replace_match, arg)
    elif isinstance(arg, list):
        return [_resolve_arg(a, observations) for a in arg]
    else:
        return str(arg)


@as_runnable
def schedule_task(task_inputs, config):
    task: Task = task_inputs["task"]
    observations: Dict[int, Any] = task_inputs["observations"]
    try:
        observation = _execute_task(task, observations, config)
    except Exception:
        import traceback

        observation = traceback.format_exception()  # repr(e) +
    observations[task["idx"]] = observation


def schedule_pending_task(
    task: Task, observations: Dict[int, Any], retry_after: float = 0.2
):
    while True:
        deps = task["dependencies"]
        if deps and (any([dep not in observations for dep in deps])):
            # Dependencies not yet satisfied
            time.sleep(retry_after)
            continue
        schedule_task.invoke({"task": task, "observations": observations})
        break


@as_runnable
def schedule_tasks(scheduler_input: SchedulerInput) -> List[FunctionMessage]:
    """Group the tasks into a DAG schedule."""
    # For streaming, we are making a few simplifying assumption:
    # 1. The LLM does not create cyclic dependencies
    # 2. That the LLM will not generate tasks with future deps
    # If this ceases to be a good assumption, you can either
    # adjust to do a proper topological sort (not-stream)
    # or use a more complicated data structure
    tasks = scheduler_input["tasks"]
    args_for_tasks = {}
    messages = scheduler_input["messages"]
    # If we are re-planning, we may have calls that depend on previous
    # plans. Start with those.
    observations = _get_observations(messages)
    task_names = {}
    originals = set(observations)
    # ^^ We assume each task inserts a different key above to
    # avoid race conditions...
    futures = []
    retry_after = 0.25  # Retry every quarter second
    with ThreadPoolExecutor() as executor:
        for task in tasks:
            deps = task["dependencies"]
            task_names[task["idx"]] = (
                task["tool"] if isinstance(task["tool"], str) else task["tool"].name
            )
            args_for_tasks[task["idx"]] = task["args"]
            if (
                # Depends on other tasks
                deps
                and (any([dep not in observations for dep in deps]))
            ):
                futures.append(
                    executor.submit(
                        schedule_pending_task, task, observations, retry_after
                    )
                )
            else:
                # No deps or all deps satisfied
                # can schedule now
                schedule_task.invoke(dict(task=task, observations=observations))
                # futures.append(executor.submit(schedule_task.invoke dict(task=task, observations=observations)))

        # All tasks have been submitted or enqueued
        # Wait for them to complete
        wait(futures)
    # Convert observations to new tool messages to add to the state
    new_observations = {
        k: (task_names[k], args_for_tasks[k], observations[k])
        for k in sorted(observations.keys() - originals)
    }
    tool_messages = [
        FunctionMessage(
            name=name, content=str(obs), additional_kwargs={"idx": k, "args": task_args}
        )
        for k, (name, task_args, obs) in new_observations.items()
    ]
    return tool_messages

In [15]:
import itertools


@as_runnable
def plan_and_schedule(messages: List[BaseMessage], config):
    tasks = planner.stream(messages, config)
    # Begin executing the planner immediately
    try:
        tasks = itertools.chain([next(tasks)], tasks)
    except StopIteration:
        # Handle the case where tasks is empty.
        tasks = iter([])
    scheduled_tasks = schedule_tasks.invoke(
        {
            "messages": messages,
            "tasks": tasks,
        },
        config,
    )
    return scheduled_tasks

#### Example Plan

We still haven't introduced any cycles in our computation graph, so this is all easily expressed in LCEL.

In [16]:
example_question

'What is the description of the latest youtube video by Krish naik on datascience?'

In [17]:
tool_messages = plan_and_schedule.invoke([HumanMessage(content=example_question)])

DATA MODEL = video_url='https://www.youtube.com/watch?v=JxgmHe2NyeY' context=None


In [18]:
tool_messages

[FunctionMessage(content="[{'url': 'https://www.youtube.com/watch?v=JxgmHe2NyeY', 'content': 'All the materials are available in the below linkhttps://github.com/krishnaik06/The-Grand-Complete-Data-Science-Materials/tree/mainVisit https://krishnaik.in...'}]", additional_kwargs={'idx': 1, 'args': {'query': 'latest youtube video by Krish Naik on datascience'}}, name='tavily_search_results_json'),
 FunctionMessage(content="{'url': 'https://www.youtube.com/watch?v=JxgmHe2NyeY', 'title': 'Sample Title', 'description': 'Sample description of the video.', 'views': '12345', 'likes': '678', 'dislikes': '90', 'comments': [{'author': 'User1', 'text': 'Sample comment 1'}, {'author': 'User2', 'text': 'Sample comment 2'}]}", additional_kwargs={'idx': 2, 'args': {'video_url': 'URL from $1'}}, name='youtube_parser'),
 FunctionMessage(content='join', additional_kwargs={'idx': 3, 'args': ()}, name='join')]

## 4. "Joiner" 

So now we have the planning and initial execution done. We need a component to process these outputs and either:

1. Respond with the correct answer.
2. Loop with a new plan.

The paper refers to this as the "joiner". It's another LLM call. We are using function calling to improve parsing reliability.

In [19]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_core.messages import AIMessage


class FinalResponse(BaseModel):
    """The final response/answer."""

    response: str


class Replan(BaseModel):
    feedback: str = Field(
        description="Analysis of the previous attempts and recommendations on what needs to be fixed."
    )


class JoinOutputs(BaseModel):
    """Decide whether to replan or whether you can return the final response."""

    thought: str = Field(
        description="The chain of thought reasoning for the selected action"
    )
    action: Union[FinalResponse, Replan]


joiner_prompt = hub.pull("wfh/llm-compiler-joiner").partial(
    examples=""
)  # You can optionally add examples
llm = ChatOpenAI(model="gpt-4o")

runnable = create_structured_output_runnable(JoinOutputs, llm, joiner_prompt)

We will select only the most recent messages in the state, and format the output to be more useful for
the planner, should the agent need to loop.

In [20]:
def _parse_joiner_output(decision: JoinOutputs) -> List[BaseMessage]:
    response = [AIMessage(content=f"Thought: {decision.thought}")]
    if isinstance(decision.action, Replan):
        return response + [
            SystemMessage(
                content=f"Context from last attempt: {decision.action.feedback}"
            )
        ]
    else:
        return response + [AIMessage(content=decision.action.response)]


def select_recent_messages(messages: list) -> dict:
    selected = []
    for msg in messages[::-1]:
        selected.append(msg)
        if isinstance(msg, HumanMessage):
            break
    return {"messages": selected[::-1]}


joiner = select_recent_messages | runnable | _parse_joiner_output

In [21]:
input_messages = [HumanMessage(content=example_question)] + tool_messages

In [22]:
joiner.invoke(input_messages)

[AIMessage(content='Thought: The description of the latest YouTube video by Krish Naik includes a link to materials on GitHub and a mention of his website.'),
 AIMessage(content='The description of the latest YouTube video by Krish Naik on data science includes a link to materials on GitHub and a mention of his website. You can find all the materials at the provided GitHub link: https://github.com/krishnaik06/The-Grand-Complete-Data-Science-Materials/tree/main and visit https://krishnaik.in for more information.')]

## 5. Compose using LangGraph

We'll define the agent as a stateful graph, with the main nodes being:

1. Plan and execute (the DAG from the first step above)
2. Join: determine if we should finish or replan
3. Recontextualize: update the graph state based on the output from the joiner

In [38]:
from langgraph.graph import MessageGraph, END
from langchain_core.agents import AgentFinish
from typing import Dict

graph_builder = MessageGraph()

# 1.  Define vertices
# We defined plan_and_schedule above already
# Assign each node to a state variable to update
graph_builder.add_node("plan_and_schedule", plan_and_schedule)
graph_builder.add_node("join", joiner)


## Define edges
graph_builder.add_edge("plan_and_schedule", "join")

### This condition determines looping logic


def should_continue(state: List[BaseMessage]):
    print("> should_continue  STATE =", state)
    if isinstance(state[-1], AIMessage) or "END" in state[-1].content:
        return END
    return "plan_and_schedule"


graph_builder.add_conditional_edges(
    "join",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)
graph_builder.set_entry_point("plan_and_schedule")
chain = graph_builder.compile()

#### Simple question

Let's ask a simple question of the agent.

In [39]:
for step in chain.stream([HumanMessage(content="What's the GDP of New York?")]):
    print(step)
    print("---")

{'plan_and_schedule': [FunctionMessage(content='[{\'url\': \'https://www.statista.com/statistics/188087/gdp-of-the-us-federal-state-of-new-york-since-1997/\', \'content\': \'Industry Overview\\nDigital & Trend reports\\nOverview and forecasts on trending topics\\nIndustry & Market reports\\nIndustry and market insights and forecasts\\nCompanies & Products reports\\nKey figures and rankings about companies and products\\nConsumer & Brand reports\\nConsumer and brand insights and preferences in various industries\\nPolitics & Society reports\\nDetailed information about political and social topics\\nCountry & Region reports\\nAll key figures about countries and regions\\nMarket forecast and expert KPIs for 1000+ markets in 190+ countries & territories\\nInsights on consumer attitudes and behavior worldwide\\nBusiness information on 100m+ public and private companies\\nExplore Company Insights\\nDetailed information for 39,000+ online stores and marketplaces\\nDirectly accessible data for

In [35]:
# Final answer
print(step)

{'plan_and_schedule': []}


#### Multi-hop question

This question requires that the agent perform multiple searches.

In [27]:
steps = chain.stream(
    [
        HumanMessage(
            content="What's the oldest parrot alive, and how much longer is that than the average?"
        )
    ],
    {
        "recursion_limit": 100,
    },
)
for step in steps:
    print(step)
    print("---")

{'plan_and_schedule': [FunctionMessage(content='[{\'url\': \'https://en.wikipedia.org/wiki/Cookie_(cockatoo)\', \'content\': \'He was one of the longest-lived birds on record[4] and was recognised by the Guinness World Records as the oldest living parrot in the world.[5]\\nThe next-oldest pink cockatoo to be found in a zoological setting was a 31-year-old female bird located at Paradise Wildlife Sanctuary, England.[3] Information published by the World Parrot Trust states longevity for Cookie\\\'s species in captivity is on average 40–60 years.[6]\\nLife[edit]\\nCookie was Brookfield Zoo\\\'s oldest resident and the last surviving member of the animal collection from the time of the zoo\\\'s opening in 1934, having arrived from Taronga Zoo of Sydney, New South Wales, Australia, in the same year and judged to be one year old at the time.[7]\\nIn the 1950s an attempt was made to introduce Cookie to a female pink cockatoo, but Cookie rejected her as "she was not nice to him".[8]\\n In 200

In [33]:
steps = chain.stream(
    [
        HumanMessage(
            content="What's the recent youtube video posted by krish naik on datascience?"
        )
    ],
    {
        "recursion_limit": 100,
    },
)
for step in steps:
    print(step)
    print("---")

{'plan_and_schedule': [FunctionMessage(content="[{'url': 'https://www.youtube.com/user/krishnaik06', 'content': 'This is my YouTube channel where I explain various topics on machine learning, deep learning, and AI with many real-world problem scenarios. ... I have delivered over 30 tech talks on data science ...'}]", additional_kwargs={'idx': 1, 'args': {'query': 'krish naik recent youtube video on datascience'}}, name='tavily_search_results_json', id='1e2e14e7-5489-4e90-9335-e784486d69b5'), FunctionMessage(content='join', additional_kwargs={'idx': 2, 'args': ()}, name='join', id='b4560ee9-d3ad-4db9-af41-c5a828945ebd')]}
---
{'join': [AIMessage(content="Thought: The search result only provides general information about Krish Naik's YouTube channel but does not give specific details about the most recent video on data science.", id='63977264-b2c1-415d-a996-ba666fce9d29'), SystemMessage(content='Context from last attempt: The search query did not return specific details about the recent 

KeyboardInterrupt: 

In [34]:
steps = chain.stream(
    [
        HumanMessage(
            content="What's mentioned in the youtube video https://www.youtube.com/playlist?list=PLYQsp-tXX9w6dCJBsgZfddg5pAuWXapU4"
        )
    ],
    {
        "recursion_limit": 100,
    },
)
for step in steps:
    print(step)
    print("---")

{'plan_and_schedule': [FunctionMessage(content='join', additional_kwargs={'idx': 1, 'args': ()}, name='join', id='5521ecbb-1cc1-4907-a55f-2ced84ec5d0f')]}
---
{'join': [AIMessage(content="Thought: I need to check the contents of the provided YouTube playlist to answer the user's question accurately.", id='f72432f1-01e3-43a1-b185-85fae9280805'), SystemMessage(content='Context from last attempt: I need to extract and review the video titles and descriptions in the YouTube playlist to provide the required information.', id='385bdc6a-2d88-4f0a-8cc8-c780281bcf79')]}
---
DATA MODEL = video_url='https://www.youtube.com/playlist?list=PLYQsp-tXX9w6dCJBsgZfddg5pAuWXapU4' context=None
{'plan_and_schedule': [FunctionMessage(content="{'url': 'https://www.youtube.com/playlist?list=PLYQsp-tXX9w6dCJBsgZfddg5pAuWXapU4', 'title': 'Sample Title', 'description': 'Sample description of the video.', 'views': '12345', 'likes': '678', 'dislikes': '90', 'comments': [{'author': 'User1', 'text': 'Sample comment 

KeyboardInterrupt: 

#### Multi-step  math

In [40]:
for step in chain.stream(
    [
        HumanMessage(
            content="What's ((3*(4+5)/0.5)+3245) + 8? What's 32/4.23? What's the sum of those two values?"
        )
    ]
):
    print(step)

{'plan_and_schedule': [FunctionMessage(content='3307.0', additional_kwargs={'idx': 1, 'args': {'problem': '3*(4+5)/0.5 + 3245 + 8'}}, name='math', id='ea6b72b7-0e21-4ae1-bcec-288be7061d83'), FunctionMessage(content='7.565011820330969', additional_kwargs={'idx': 2, 'args': {'problem': '32/4.23'}}, name='math', id='ef240d3c-baee-4125-8f63-df462ea746b5'), FunctionMessage(content='3314.565011820331', additional_kwargs={'idx': 3, 'args': {'problem': 'sum of $1 and $2', 'context': ['$1', '$2']}}, name='math', id='cf7ff256-3012-4c60-b7b6-7451ff2cbd15'), FunctionMessage(content='join', additional_kwargs={'idx': 4, 'args': ()}, name='join', id='41e4745e-20f3-4227-8519-e69fe5e857df')]}
> should_continue  STATE = [HumanMessage(content="What's ((3*(4+5)/0.5)+3245) + 8? What's 32/4.23? What's the sum of those two values?", id='c0d5d967-cef5-4c82-be3d-4fe9cc347229'), FunctionMessage(content='3307.0', additional_kwargs={'idx': 1, 'args': {'problem': '3*(4+5)/0.5 + 3245 + 8'}}, name='math', id='ea6b72

In [45]:
# Final answer
print(step["join"][-1].content)

The value of ((3*(4+5)/0.5)+3245) + 8 is 3307.0. The value of 32/4.23 is 7.565011820330969. The sum of those two values is 3314.565011820331.


## Conclusion

Congrats on building your first LLMCompiler agent! I'll leave you with some known limitations to the implementation above:

1. The planner output parsing format is fragile if your function requires more than 1 or 2 arguments. We could make it more robust by using streaming tool calling.
2. Variable substitution is fragile in the example above. It could be made more robust by using a fine-tuned model and a more robust syntax (using e.g., Lark or a tool calling schema)
3. The state can grow quite long if you require multiple re-planning runs. To handle, you could add a message compressor once you go above a certain token limit.
